In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 

# sk learn import 
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier


C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train_1 = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\AibHack\data\training.txt", sep="\t", encoding = 'iso-8859-1')
test_1 = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\AibHack\data\testing.txt", sep="\t", encoding = 'iso-8859-1')

In [24]:
#Join the datasets
train = train_1.append(test_1,ignore_index = True)

In [25]:
train_2 = train[pd.notnull(train['EnergyRatingCat'])]

In [26]:
cats = []
for col in train.columns.values:
    if train[col].dtype == 'object':
        cats.append(col)

In [27]:
df_cont = train.drop(cats, axis=1)
df_cat = train[cats]


### Feature Engineering

In [28]:
train.loc[:, "SecSHFrac"] = train.loc[:, "SecSHFrac"].fillna(train['SecSHFrac'].median())
train.loc[:, "SecSHEfficiency"] = train.loc[:, "SecSHEfficiency"].fillna(train['SecSHEfficiency'].median())
train.loc[:, "MainWHEfficiency"] = train.loc[:, "MainWHEfficiency"].fillna(train['MainWHEfficiency'].median())
train.loc[:, "SecSHEfficiency"] = train.loc[:, "SecSHEfficiency"].fillna(train['SecSHEfficiency'].median())
train.loc[:, "SecSHFrac"] = train.loc[:, "SecSHFrac"].fillna(train['SecSHFrac'].median())
train.loc[:, "MainWHAdj"] = train.loc[:, "MainWHAdj"].fillna(train['MainWHAdj'].median())
train.loc[:, "SecSHFuel"] = train.loc[:, "SecSHFuel"].fillna(train['SecSHFuel'].median())
train.loc[:, "SecWHFuel"] = train.loc[:, "SecWHFuel"].fillna(train['SecWHFuel'].median())
train.loc[:, "SecSHFrac"] = train.loc[:, "SecSHFrac"].fillna(train['SecSHFrac'].median())
train.loc[:, "TempAdj"] = train.loc[:, "TempAdj"].fillna(train['TempAdj'].median())


In [29]:
#drop columns
del train['StorageLoss']
del train['DeclaredLoss']
del train['SolarWaterHeating']
del train['SummerElectricImm']
del train['Combi']
del train['KeepHot']
del train['VolWaterStorage']
del train['TempFac']
del train['TempFacMult']
del train['HWInsulationType']
del train['HWInsulationThickness']
del train['AreaAperture']
del train['ZeroLossCollEff']
del train['CollHeatLoss']

In [30]:
#drop columns 458880
del train['SolarRadiation']
del train['Overshading']
del train['DedicatedSolarVol']
del train['SolarPumpElect']
del train['SolarHeatFrac']
del train['LargestRoofTypeArea']
del train['LargestRoofType']
del train['SecSpacePrimaryEnergy']
del train['FuelTypeFirstBoiler']
del train['EffFirstHeatGenPlant']
del train['PercFirstHeat']
del train['FuelTypeSecondBoiler']
del train['FuelTypeThirdBoiler']
del train['EffThirdHeatGenPlant']
del train['PercThirdHeat']
del train['HeatingSystemSolarSpace']
#del train['TotPrimaryEnergyFac']

In [31]:
train.loc[:, "FirstEnerType"] = train.loc[:, "FirstEnerType"].fillna("None")
train.loc[:, "FirstEnerTypeDescr"] = train.loc[:, "FirstEnerTypeDescr"].fillna("None")
train.loc[:, "FirstEnerType"] = train.loc[:, "FirstEnerType"].fillna("None")
train.loc[:, "FirstEnerTypeDescr"] = train.loc[:, "FirstEnerTypeDescr"].fillna("None")
train.loc[:, "FirstEnerType"] = train.loc[:, "FirstEnerType"].fillna("None")
train.loc[:, "FirstEnerTypeDescr"] = train.loc[:, "FirstEnerTypeDescr"].fillna("None")

train.loc[:, "PercLowEnergy"] = train.loc[:, "PercLowEnergy"].fillna("No")
train.loc[:, "ProdSecondEnerConvFac"] = train.loc[:, "ProdSecondEnerConvFac"].fillna("No")
train.loc[:, "ConsSecondEnerComment"] = train.loc[:, "ConsSecondEnerComment"].fillna("No")
train.loc[:, "SecondEnerTypeDescr"] = train.loc[:, "SecondEnerTypeDescr"].fillna("No")
train.loc[:, "ProdSecondEnerComment"] = train.loc[:, "ProdSecondEnerComment"].fillna("No")
train.loc[:, "ProdSecondEnerDelivered"] = train.loc[:, "ProdSecondEnerDelivered"].fillna("No")

#train.loc[:, "AvgWallU"] = train.loc[:, "AvgWallU"].fillna(0)
train.loc[:, "ProdFirstEnerComment"] = train.loc[:, "ProdFirstEnerComment"].fillna("No")
train.loc[:, "ProdFirstEnerDelivered"] = train.loc[:, "ProdFirstEnerDelivered"].fillna("No")
train.loc[:, "ProdFirstEnerConvFac"] = train.loc[:, "ProdFirstEnerConvFac"].fillna("No")
train.loc[:, "ConsFirstEnerComment"] = train.loc[:, "ConsFirstEnerComment"].fillna("No")
#train.loc[:, "MainSHEfficiency"] = train.loc[:, "MainSHEfficiency"].fillna("No")
train.loc[:, "PrimaryMainWaterEnergy"] = train.loc[:, "PrimaryMainWaterEnergy"].fillna(train['PrimaryMainWaterEnergy'].median())
train.loc[:, "PrimaryMainSpaceEnergy"] = train.loc[:, "PrimaryMainSpaceEnergy"].fillna(train['PrimaryMainSpaceEnergy'].median())
train.loc[:, "MainWaterCO2"] = train.loc[:, "MainWaterCO2"].fillna(train['MainWaterCO2'].median())
train.loc[:, "MainSpaceCO2"] = train.loc[:, "MainSpaceCO2"].fillna(train['MainSpaceCO2'].median())
train.loc[:, "SecSHFrac"] = train.loc[:, "SecSHFrac"].fillna(train['SecSHFrac'].median())
#train.loc[:, "PercLowEnergy"] = train.loc[:, "PercLowEnergy"].fillna(train['PercLowEnergy'].median())
#train.loc[:, "SecSpaceEnergy"] = train.loc[:, "SecSpaceEnergy"].fillna(train['SecSpaceEnergy'].median())
train.loc[:, "SecWaterEnergy"] = train.loc[:, "SecWaterEnergy"].fillna(train['SecWaterEnergy'].median())
train.loc[:, "SecSHFrac"] = train.loc[:, "SecSHFrac"].fillna(train['SecSHFrac'].median())
train.loc[:, "TempAdj"] = train.loc[:, "TempAdj"].fillna(train['TempAdj'].median())
train.loc[:, "TotPrimaryEnergyFac"] = train.loc[:, "TotPrimaryEnergyFac"].fillna(train['TotPrimaryEnergyFac'].median())

In [32]:
train.loc[:, "ConsThirdEnerComment"] = train.loc[:, "ConsThirdEnerComment"].fillna("No")
#train.loc[:, "ProdThirdEnerConvFac"] = train.loc[:, "ProdThirdEnerConvFac"].fillna("No")


train.loc[:, "SecSpaceEnergy"] = train.loc[:, "SecSpaceEnergy"].fillna("No")
train.loc[:, "ThirdEnerTypeDescr"] = train.loc[:, "ThirdEnerTypeDescr"].fillna("No")
train.loc[:, "ProdThirdEnerComment"] = train.loc[:, "ProdThirdEnerComment"].fillna("No")
train.loc[:, "ProdThirdEnerDelivered"] = train.loc[:, "ProdThirdEnerDelivered"].fillna("No")
train.loc[:, "ProdThirdEnerConvFac"] = train.loc[:, "ProdThirdEnerConvFac"].fillna("No")
#train.loc[:, "AvgWallU"] = train.loc[:, "AvgWallU"].fillna(0)
train.loc[:, "TotCO2Emissions"] = train.loc[:, "TotCO2Emissions"].fillna(train['TotCO2Emissions'].median())
train.loc[:, "PrimaryEnergySecondarySpace"] = train.loc[:, "PrimaryEnergySecondarySpace"].fillna(train['TotCO2Emissions'].median())

In [33]:
train.loc[:, "TotEnergy"] = train.loc[:, "TotEnergy"].fillna('No')
#train.loc[:, "TotEnergy"] = train.loc[:, "TotEnergy"].fillna(train['TotEnergy'].median())
train.loc[:, "SecSpaceCO2"] = train.loc[:, "SecSpaceCO2"].fillna(train['SecSpaceCO2'].median())

In [34]:
train.loc[:, "ConsThirdEnerComment"] = train.loc[:, "ConsThirdEnerComment"].fillna("No")
#train.loc[:, "ProdThirdEnerConvFac"] = train.loc[:, "ProdThirdEnerConvFac"].fillna("No")


train.loc[:, "SecSpaceEnergy"] = train.loc[:, "SecSpaceEnergy"].fillna("No")
train.loc[:, "ThirdEnerTypeDescr"] = train.loc[:, "ThirdEnerTypeDescr"].fillna("No")
train.loc[:, "ProdThirdEnerComment"] = train.loc[:, "ProdThirdEnerComment"].fillna("No")
train.loc[:, "ProdThirdEnerDelivered"] = train.loc[:, "ProdThirdEnerDelivered"].fillna("No")
train.loc[:, "ProdThirdEnerConvFac"] = train.loc[:, "ProdThirdEnerConvFac"].fillna("No")
#train.loc[:, "AvgWallU"] = train.loc[:, "AvgWallU"].fillna(0)
train.loc[:, "TotCO2Emissions"] = train.loc[:, "TotCO2Emissions"].fillna(train['TotCO2Emissions'].median())
train.loc[:, "PrimaryEnergySecondarySpace"] = train.loc[:, "PrimaryEnergySecondarySpace"].fillna(train['TotCO2Emissions'].median())

In [35]:
train.loc[:, "MainSHFuel"] = train.loc[:, "MainSHFuel"].fillna("No")
train.loc[:, "MainWHFuel"] = train.loc[:, "MainWHFuel"].fillna("No")
train.loc[:, "PrimaryLoss"] = train.loc[:, "PrimaryLoss"].fillna("No")
#train.loc[:, "AvgWallU"] = train.loc[:, "AvgWallU"].fillna(0)
train.loc[:, "AvgWallU"] = train.loc[:, "AvgWallU"].fillna(train['AvgWallU'].median())
#train.loc[:, "MainSHFuel"] = train.loc[:, "MainSHFuel"].fillna(train['MainSHFuel'].median())
#train.loc[:, "MainWHFuel"] = train.loc[:, "MainWHFuel"].fillna(train['MainWHFuel'].median())
train.loc[:, "MainSHEfficiency"] = train.loc[:, "MainSHEfficiency"].fillna(train['MainSHEfficiency'].median())
train.loc[:, "MainSHAdj"] = train.loc[:, "MainSHAdj"].fillna(train['MainSHAdj'].median())
#train.loc[:, "PrimaryLoss"] = train.loc[:, "PrimaryLoss"].fillna(train['PrimaryLoss'].median())
train.loc[:, "EffSecondHeatGenPlant"] = train.loc[:, "EffSecondHeatGenPlant"].fillna(train['EffSecondHeatGenPlant'].median())
train.loc[:, "PercSecondHeat"] = train.loc[:, "PercSecondHeat"].fillna(train['PercSecondHeat'].median())

In [38]:
target = train[['EnergyRatingCat','EnergyRatingCont']]

In [40]:

data_cl = train.copy() # create a copy of data frame
target = data_cl[['EnergyRatingCat','EnergyRatingCont']].copy()

In [36]:
import re
missing_values = []
nonumeric_values = []

print ("TRAIN SET INFORMATION")
print ("========================\n")

for column in train:
    # Find all the unique feature values
    uniq = train[column].unique()
    print ("'{}' has {} unique values" .format(column,uniq.size))
    if (uniq.size > 17):
        print("~~Listing up to 17 unique values~~")
    print (uniq[0:17])
    print ("\n-----------------------------------------------------------------------\n")
    
    # Find features with missing values
    if (True in pd.isnull(uniq)):
        s = "{} has {} missing" .format(column, pd.isnull(train[column]).sum())
        missing_values.append(s)
    
    # Find features with non-numeric values
    for i in range (1, np.prod(uniq.shape)):
        if (re.match('nan', str(uniq[i]))):
            break
        if not (re.search('(^\d+\.?\d*$)|(^\d*\.?\d+$)', str(uniq[i]))):
            nonumeric_values.append(column)
            break
  
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print ("Features with missing values:\n{}\n\n" .format(missing_values))
print ("Features with non-numeric values:\n{}" .format(nonumeric_values))
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

TRAIN SET INFORMATION

'BuildingID' has 623776 unique values
~~Listing up to 17 unique values~~
[190835 523113 271393 585961 624083  30231 350444 592196 365924 303002
 634938 300824 449623 379988  68299 597104 163298]

-----------------------------------------------------------------------

'County' has 56 unique values
~~Listing up to 17 unique values~~
['Co. Wexford' 'Co. Clare' 'Co. Kerry' 'Co. Cork' 'Co. Laois' 'Cork City'
 'Co. Carlow' 'Co. Offaly' 'Co. Donegal' 'Co. Meath' 'Co. Cavan' 'Dublin 4'
 'Co. Galway' 'Dublin 14' 'Dublin 6' 'Co. Roscommon' 'Dublin 17']

-----------------------------------------------------------------------

'Type' has 12 unique values
['Mid-terrace house' 'Detached house' 'Semi-detached house'
 'Ground-floor apartment' 'Mid-floor apartment' 'End of terrace house'
 'Top-floor apartment' 'House' 'Maisonette' 'Apartment' 'Basement Dwelling'
 nan]

-----------------------------------------------------------------------

'Year' has 253 unique values
~~Listing

In [16]:
train.loc[:, "County"] = train.loc[:, "County"].fillna("None")
train.loc[:, "Type"] = train.loc[:, "Type"].fillna("None")
train.loc[:, "RatingType"] = train.loc[:, "RatingType"].fillna("None")
train.loc[:, "FirstEnerTypeDescr"] = train.loc[:, "FirstEnerTypeDescr"].fillna("None")
train.loc[:, "FirstEnerType"] = train.loc[:, "FirstEnerType"].fillna("None")
train.loc[:, "FirstEnerTypeDescr"] = train.loc[:, "FirstEnerTypeDescr"].fillna("None")

train.loc[:, "PercLowEnergy"] = train.loc[:, "PercLowEnergy"].fillna("No")
train.loc[:, "ProdSecondEnerConvFac"] = train.loc[:, "ProdSecondEnerConvFac"].fillna("No")
train.loc[:, "ConsSecondEnerComment"] = train.loc[:, "ConsSecondEnerComment"].fillna("No")
train.loc[:, "SecondEnerTypeDescr"] = train.loc[:, "SecondEnerTypeDescr"].fillna("No")
train.loc[:, "ProdSecondEnerComment"] = train.loc[:, "ProdSecondEnerComment"].fillna("No")
train.loc[:, "ProdSecondEnerDelivered"] = train.loc[:, "ProdSecondEnerDelivered"].fillna("No")

#train.loc[:, "AvgWallU"] = train.loc[:, "AvgWallU"].fillna(0)
train.loc[:, "ProdFirstEnerComment"] = train.loc[:, "ProdFirstEnerComment"].fillna("No")
train.loc[:, "ProdFirstEnerDelivered"] = train.loc[:, "ProdFirstEnerDelivered"].fillna("No")
train.loc[:, "ProdFirstEnerConvFac"] = train.loc[:, "ProdFirstEnerConvFac"].fillna("No")
train.loc[:, "ConsFirstEnerComment"] = train.loc[:, "ConsFirstEnerComment"].fillna("No")
#train.loc[:, "MainSHEfficiency"] = train.loc[:, "MainSHEfficiency"].fillna("No")
train.loc[:, "PrimaryMainWaterEnergy"] = train.loc[:, "PrimaryMainWaterEnergy"].fillna(train['PrimaryMainWaterEnergy'].median())
train.loc[:, "PrimaryMainSpaceEnergy"] = train.loc[:, "PrimaryMainSpaceEnergy"].fillna(train['PrimaryMainSpaceEnergy'].median())
train.loc[:, "MainWaterCO2"] = train.loc[:, "MainWaterCO2"].fillna(train['MainWaterCO2'].median())
train.loc[:, "MainSpaceCO2"] = train.loc[:, "MainSpaceCO2"].fillna(train['MainSpaceCO2'].median())
train.loc[:, "SecSHFrac"] = train.loc[:, "SecSHFrac"].fillna(train['SecSHFrac'].median())

NameError: name 'df_cat' is not defined

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
class_le = LabelEncoder()

In [19]:
train['County'] = class_le.fit_transform(train['County'].values)
train['Type'] = class_le.fit_transform(train['Type'].values)
train['RatingType'] = class_le.fit_transform(train['RatingType'].values)
train['EnergyRatingCat'] = class_le.fit_transform(train['EnergyRatingCat'].values)
train['GroundFloorArea'] = class_le.fit_transform(train['GroundFloorArea'].values)
train['MainSHFuel'] = class_le.fit_transform(train['MainSHFuel'].values)
train['MainWHFuel'] = class_le.fit_transform(train['MainWHFuel'].values)
train['SharedMPRN'] = class_le.fit_transform(train['SharedMPRN'].values)
train['DLobby'] = class_le.fit_transform(train['DLobby'].values)
train['VentMethod'] = class_le.fit_transform(train['VentMethod'].values)


TypeError: unorderable types: str() > float()

In [20]:
train['WallStructure'] = class_le.fit_transform(train['WallStructure'].values)
train['SuspWoodFloor'] = class_le.fit_transform(train['SuspWoodFloor'].values)
train['PermTest'] = class_le.fit_transform(train['PermTest'].values)
train['BoilerStat1'] = class_le.fit_transform(train['BoilerStat1'].values)
train['BoilerStat2'] = class_le.fit_transform(train['BoilerStat2'].values)
train['OilPumpIn'] = class_le.fit_transform(train['OilPumpIn'].values)
train['WarmAir'] = class_le.fit_transform(train['WarmAir'].values)
train['Underfloor'] = class_le.fit_transform(train['Underfloor'].values)
#train['StorageLoss'] = class_le.fit_transform(train['StorageLoss'].values)
#train['DeclaredLoss'] = class_le.fit_transform(train['DeclaredLoss'].values)

TypeError: unorderable types: str() > float()

In [21]:
cats_1 = []
for col in train.columns.values:
    if train[col].dtype == 'object':
        cats_1.append(col)

In [22]:
df_cat_1 = train[cats_1]


In [23]:
df_cat_1.columns

Index(['County', 'Type', 'Year', 'RatingType', 'EnergyRatingCat',
       'GroundFloorArea', 'MainSHFuel', 'MainWHFuel', 'SharedMPRN', 'DLobby',
       'VentMethod', 'WallStructure', 'SuspWoodFloor', 'PermTest',
       'BoilerStat1', 'BoilerStat2', 'OilPumpIn', 'WarmAir', 'Underfloor',
       'PrimaryLoss', 'CylinderThermostat', 'CombiCylinder', 'SolarPumpPower',
       'Charging', 'MainSpaceEnergy', 'MassCat', 'PercLowEnergy', 'TotEnergy',
       'SecSpaceEnergy', 'PercLivingArea', 'RatingPurpose', 'AssessmentDate',
       'FirstEnerType', 'FirstEnerTypeDescr', 'ProdFirstEnerComment',
       'ProdFirstEnerDelivered', 'ProdFirstEnerConvFac',
       'ConsFirstEnerComment', 'SecondEnerTypeDescr', 'ProdSecondEnerComment',
       'ProdSecondEnerDelivered', 'ProdSecondEnerConvFac',
       'ConsSecondEnerComment', 'ThirdEnerTypeDescr', 'ProdThirdEnerComment',
       'ProdThirdEnerDelivered', 'ProdThirdEnerConvFac',
       'ConsThirdEnerComment'],
      dtype='object')

In [46]:
train['Year'] = class_le.fit_transform(train['Year'].values)
train['Charging'] = class_le.fit_transform(train['Charging'].values)
train['MainSpaceEnergy'] = class_le.fit_transform(train['MainSpaceEnergy'].values)
train['MassCat'] = class_le.fit_transform(train['MassCat'].values)
#train['TotEnergy'] = class_le.fit_transform(train['TotEnergy'].values)
#train['SecSpaceEnergy'] = class_le.fit_transform(train['SecSpaceEnergy'].values)
train['PercLivingArea'] = class_le.fit_transform(train['PercLivingArea'].values)
train['RatingPurpose'] = class_le.fit_transform(train['RatingPurpose'].values)
train['PrimaryLoss'] = class_le.fit_transform(train['PrimaryLoss'].values)
#train['DeclaredLoss'] = class_le.fit_transform(train['DeclaredLoss'].values)

In [55]:
train['AssessmentDate'] = class_le.fit_transform(train['AssessmentDate'].values)
#train['FirstEnerType'] = class_le.fit_transform(train['FirstEnerType'].values)
train['FirstEnerTypeDescr'] = class_le.fit_transform(train['FirstEnerTypeDescr'].values)
#train['ProdFirstEnerDelivered'] = class_le.fit_transform(train['ProdFirstEnerDelivered'].values)
#train['ProdFirstEnerComment'] = class_le.fit_transform(train['ProdFirstEnerComment'].values)
#train['ProdFirstEnerConvFac'] = class_le.fit_transform(train['ProdFirstEnerConvFac'].values)
#train['ConsFirstEnerComment'] = class_le.fit_transform(train['ConsFirstEnerComment'].values)
train['SecondEnerTypeDescr'] = class_le.fit_transform(train['SecondEnerTypeDescr'].values)
#train['ProdSecondEnerComment'] = class_le.fit_transform(train['ProdSecondEnerComment'].values)
#train['DeclaredLoss'] = class_le.fit_transform(train['DeclaredLoss'].values)

In [63]:
#train['ProdSecondEnerDelivered'] = class_le.fit_transform(train['ProdSecondEnerDelivered'].values)
#train['ProdSecondEnerConvFac'] = class_le.fit_transform(train['ProdSecondEnerConvFac'].values)
#train['ConsSecondEnerComment'] = class_le.fit_transform(train['ConsSecondEnerComment'].values)
train['MassCat'] = class_le.fit_transform(train['MassCat'].values)
train['ThirdEnerTypeDescr'] = class_le.fit_transform(train['ThirdEnerTypeDescr'].values)
#train['SecSpaceEnergy'] = class_le.fit_transform(train['SecSpaceEnergy'].values)
#train['ProdThirdEnerComment'] = class_le.fit_transform(train['ProdThirdEnerComment'].values)
#train['ProdThirdEnerDelivered'] = class_le.fit_transform(train['ProdThirdEnerDelivered'].values)
#train['ProdThirdEnerDelivered'] = class_le.fit_transform(train['ProdThirdEnerDelivered'].values)
#train['ConsThirdEnerComment'] = class_le.fit_transform(train['ConsThirdEnerComment'].values)

In [64]:
import re
missing_values = []
nonumeric_values = []

print ("TRAIN SET INFORMATION")
print ("========================\n")

for column in train:
    # Find all the unique feature values
    uniq = train[column].unique()
    print ("'{}' has {} unique values" .format(column,uniq.size))
    if (uniq.size > 17):
        print("~~Listing up to 17 unique values~~")
    print (uniq[0:17])
    print ("\n-----------------------------------------------------------------------\n")
    
    # Find features with missing values
    if (True in pd.isnull(uniq)):
        s = "{} has {} missing" .format(column, pd.isnull(train[column]).sum())
        missing_values.append(s)
    
    # Find features with non-numeric values
    for i in range (1, np.prod(uniq.shape)):
        if (re.match('nan', str(uniq[i]))):
            break
        if not (re.search('(^\d+\.?\d*$)|(^\d*\.?\d+$)', str(uniq[i]))):
            nonumeric_values.append(column)
            break
  
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print ("Features with missing values:\n{}\n\n" .format(missing_values))
print ("Features with non-numeric values:\n{}" .format(nonumeric_values))
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

TRAIN SET INFORMATION

'BuildingID' has 464502 unique values
~~Listing up to 17 unique values~~
[190835 523113 271393 585961 624083  30231 350444 592196 365924 303002
 634938 300824 449623 379988  68299 597104 163298]

-----------------------------------------------------------------------

'County' has 55 unique values
~~Listing up to 17 unique values~~
[24  2  7  3 10 26  0 18  4 16  1 45  6 32 47 19 35]

-----------------------------------------------------------------------

'Type' has 11 unique values
[ 8  2  9  4  7  3 10  5  6  0  1]

-----------------------------------------------------------------------

'Year' has 248 unique values
~~Listing up to 17 unique values~~
[121 132 199 233 238 234 231 220 131 211 232 221 181 236 218 196 230]

-----------------------------------------------------------------------

'RatingType' has 3 unique values
[0 1 2]

-----------------------------------------------------------------------

'EnergyRatingCat' has 15 unique values
[13 14 10  7  5  

In [41]:
# Separate dataset for validation
unknown_mask = train[['EnergyRatingCat','EnergyRatingCont']].isnull()
data_submit = data_cl[unknown_mask]

# Separate dataset for training
X = data_cl[~unknown_mask]
Y = target[~unknown_mask]

In [50]:
X['EnergyRatingCat'] = X['EnergyRatingCat'].replace({"Nan" : -1, "A1" : 15 , "A3" : 13, "A2" : 14,"B1" : 12, "B2" : 11, "B3" : 10, "C1" : 9 , "C2" : 8, "C3" : 7,"D1" : 6, "D2" : 5,"D3" : 4, "E1" : 3 , "E2" : 2, "F" : 1,"G" : 0})

In [46]:
Y

,EnergyRatingCat,EnergyRatingCont
11708,NaN,NaN
23821,NaN,NaN
61157,NaN,NaN
82063,NaN,NaN
111091,NaN,NaN
140098,NaN,NaN
144394,NaN,NaN
202716,NaN,NaN
213631,NaN,NaN
221437,NaN,NaN


In [51]:
#Variance Threshold
#Find all features with more than 90% variance in values.
threshold = 0.90
vt = VarianceThreshold().fit(X)

# Find feature names
feat_var_threshold = data_cl.columns[vt.variances_ > threshold * (1-threshold)]
feat_var_threshold

ValueError: could not convert string to float: 'F '